## Imports & Configuration

In [ ]:
import os
import requests
import time
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

# Define the directory to save ALL raw data
# We use 'raw_full' to distinguish it from manual downloads
DATA_DIR = os.path.join(os.getcwd(), "data", "raw_full")
os.makedirs(DATA_DIR, exist_ok=True)

# CDC NHANES Base URL
BASE_URL = "https://wwwn.cdc.gov"

# The 5 main components of NHANES data structure
# The script will iterate through these pages to find ALL datasets.
COMPONENTS = [
    "Demographics",
    "Dietary",
    "Examination",
    "Laboratory",
    "Questionnaire"
]

# Target Cycle
CYCLE_YEAR = "2017-2018"
CYCLE_BEGIN_YEAR = "2017"

print(f“Environment Ready.")
print(f"Target Directory: {DATA_DIR}")
print(f"Target Cycle: {CYCLE_YEAR}")

ModuleNotFoundError: No module named 'pandas'

## Crawler Functions

In [ ]:
def get_component_url(component):
    """
    Generates the URL for a specific NHANES component page.
    """
    return f"https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component={component}&CycleBeginYear={CYCLE_BEGIN_YEAR}"

def get_all_xpt_links(component_url):
    """
    Scrapes the CDC page to find ALL links ending in .XPT
    Returns a set of unique URLs.
    """
    print(f"🔎 Scanning page: {component_url}")
    try:
        response = requests.get(component_url, timeout=30)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        xpt_links = set() # Use set to avoid duplicates

        # Find all <a> tags
        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']
            # Check if it is an XPT file (case insensitive)
            if href.lower().strip().endswith('.xpt'):
                # Convert relative path to full URL
                full_url = urljoin(BASE_URL, href)
                xpt_links.add(full_url)
        
        return list(xpt_links)

    except Exception as e:
        print(f"❌ Error scanning page: {e}")
        return []

def download_file(url):
    """
    Downloads a single file from the URL.
    """
    filename = url.split('/')[-1]
    save_path = os.path.join(DATA_DIR, filename)

    # Check if file already exists to avoid re-downloading
    if os.path.exists(save_path):
        print(f"   Using cached: {filename}")
        return True

    print(f"   ⬇️ Downloading: {filename} ...")
    try:
        # Stream download for large files
        with requests.get(url, stream=True, timeout=60) as r:
            r.raise_for_status()
            with open(save_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        
        # Polite delay
        time.sleep(0.5) 
        return True
        
    except Exception as e:
        print(f"   ⚠️ Failed to download {filename}: {e}")
        return False

## Execution - Download Everything

In [ ]:
print(f"🚀 STARTING FULL AUTOMATED DOWNLOAD FOR NHANES {CYCLE_YEAR}")
print("="*60)

total_files_found = 0
total_downloaded = 0

for component in COMPONENTS:
    print(f"\n[PHASE] Processing Component: {component.upper()}")
    
    # 1. Get URL
    url = get_component_url(component)
    
    # 2. Find all datasets dynamically
    links = get_all_xpt_links(url)
    count = len(links)
    total_files_found += count
    
    print(f"   -> Found {count} datasets.")
    
    # 3. Download each one
    for link in links:
        if download_file(link):
            total_downloaded += 1

print("\n" + "="*60)
print("🎉 DOWNLOAD COMPLETE")
print(f"📊 Total Datasets Found: {total_files_found}")
print(f"📂 Location: {DATA_DIR}")
print("="*60)

## Verification

In [ ]:
import glob

# List all XPT files in the directory
all_files = glob.glob(os.path.join(DATA_DIR, "*.xpt")) + glob.glob(os.path.join(DATA_DIR, "*.XPT"))

print(f"✅ Verified: {len(all_files)} files represent in '{DATA_DIR}'")
print("\nSample files:")
for f in all_files[:5]:
    print(f" - {os.path.basename(f)}")